# Import dependencies

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from safetensors.torch import save_file

# Create dataset

In [18]:
scaler = MinMaxScaler()
X, y = make_classification(n_samples=100_000, n_features=20, n_informative=18, n_redundant=2, n_repeated=0, n_classes=2, random_state=42)
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

cuda


# Model


In [19]:
class Model(nn.Module):
    def __init__(self, hidden_layer=64):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(20, hidden_layer)
        self.fc2 = nn.Linear(hidden_layer, hidden_layer)
        self.fc3 = nn.Linear(hidden_layer, hidden_layer)
        self.fc4 = nn.Linear(hidden_layer, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

In [20]:
GRID_CONFIG = {
  'hidden_layer': [64, 128, 512],
  'learning_rate': [0.01, 0.005, 0.001],
}

EPOCH = 30

best_model = None
best_accuracy = 0
best_params = {}

# Train & Test

In [21]:
for hidden_layer in GRID_CONFIG['hidden_layer']:
    for lr in GRID_CONFIG['learning_rate']:
        # Create model
        model = Model(hidden_layer=hidden_layer).to(device)

        # Loss function and optimizer
        criterion = nn.BCELoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr)

        # Train model
        model.train()
        for epoch in range(EPOCH):
            optimizer.zero_grad()
            y_pred = model(X_train_tensor).squeeze()
            loss = criterion(y_pred, y_train_tensor)
            loss.backward()
            optimizer.step()

        # Evaluate
        model.eval()
        with torch.no_grad():
            y_pred_test = model(X_test_tensor).squeeze()
            y_pred_test = (y_pred_test > 0.5).float()  # Convert sigmoid output to binary (0 or 1)

        # Calculate accuracy
        accuracy = accuracy_score(y_test_tensor.cpu(), y_pred_test.cpu())
        print(f"Hidden Layer: {hidden_layer:>4}   Learning Rate: {lr:<7} Accuracy: {accuracy:<5.4f}")

        # Track the best model based on accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_params = {'hidden_layer': hidden_layer, 'learning_rate': lr}

print(f"Best Model Parameters: {best_params}, Best Accuracy: {best_accuracy}")

Hidden Layer:   64   Learning Rate: 0.01    Accuracy: 0.7928
Hidden Layer:   64   Learning Rate: 0.005   Accuracy: 0.8085
Hidden Layer:   64   Learning Rate: 0.001   Accuracy: 0.7471
Hidden Layer:  128   Learning Rate: 0.01    Accuracy: 0.8239
Hidden Layer:  128   Learning Rate: 0.005   Accuracy: 0.8102
Hidden Layer:  128   Learning Rate: 0.001   Accuracy: 0.7637
Hidden Layer:  512   Learning Rate: 0.01    Accuracy: 0.7690
Hidden Layer:  512   Learning Rate: 0.005   Accuracy: 0.8209
Hidden Layer:  512   Learning Rate: 0.001   Accuracy: 0.8469
Best Model Parameters: {'hidden_layer': 512, 'learning_rate': 0.001}, Best Accuracy: 0.8469333333333333


# Save best model

In [22]:
if best_model:
    torch.save({
        'X_test': X_test_tensor,
        'y_test': y_test_tensor
    }, "test_data.pt")
    save_file(best_model.state_dict(), "best_model.safetensors")